In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use('ggplot')
from keras.datasets import mnist
import time

In [ ]:
# load and preprocess labels
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = y_train.astype('int')
y_test = y_test.astype('int')


# create two classes with label -1 for even and 1 for odd
for i,j in enumerate(y_train):
    if (j % 2 == 0):
        y_train[i] = -1
    else:
        y_train[i] = 1

for i,j in enumerate(y_test):
    if (j % 2 == 0):
        y_test[i] = -1
    else:
        y_test[i] = 1


In [ ]:
# preprocess data(change shape and normalize)

# x_test = x_test.astype('float64')
# x_train = x_train.astype('float64')

x_train = x_train.reshape(x_train.shape[0],784)
x_test = x_test.reshape(x_test.shape[0],784)
x_train = x_train / 255

x_test = x_test / 255

In [ ]:
x_train = x_train[:3000]
y_train = y_train[:3000]

x_test = x_test[:500]
y_test = y_test[:500]

In [ ]:
# Support Vecrtor Machine implementation using cvxopt for optimization
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers

# the kernels that will be used
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

class SupportVectorMachine(object):

    def __init__(self, kernel=linear_kernel, C=1):
        self.kernel = kernel
        self.C = C
        

    def fit(self, X, y):
        n_samples, n_features = X.shape

        #the kernel matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = cvxopt.matrix(A, (1, n_samples), 'd')
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # soution of quadratic problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        #extract Lagrange multipliers
        a = np.ravel(solution['x'])

        #find non zero lagrange multipliers that correspond to support vectors 
        sv = a > 1e-7
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        
        # calculate the bias
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector if the linear kernel is used
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None
    # find the desicion function
    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b
    # find the sign of the decision function
    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
#method to find the accuracies
def calc_percentage(y_pre, y):
  count = 0
  for i in range(y_pre.shape[0]):
    if y_pre[i] == y[i]:
        count += 1
  count = count / y_pre.shape[0]
  return count



In [ ]:
# fit svm class
start = time.time()
svm = SupportVectorMachine(C=1,kernel=gaussian_kernel)
svm.fit(x_train, y_train)

end = time.time()
print(end - start)

     pcost       dcost       gap    pres   dres
 0: -3.5364e+02 -5.5675e+03  2e+04  2e+00  5e-15
 1: -2.6065e+02 -3.1365e+03  4e+03  2e-01  5e-15
 2: -2.9528e+02 -7.7064e+02  5e+02  2e-02  5e-15
 3: -3.4808e+02 -5.2578e+02  2e+02  5e-03  5e-15
 4: -3.7030e+02 -4.5765e+02  9e+01  2e-03  5e-15
 5: -3.8501e+02 -4.1220e+02  3e+01  3e-04  6e-15
 6: -3.9106e+02 -3.9714e+02  6e+00  7e-06  6e-15
 7: -3.9281e+02 -3.9354e+02  7e-01  4e-07  6e-15
 8: -3.9307e+02 -3.9309e+02  2e-02  1e-08  6e-15
 9: -3.9308e+02 -3.9308e+02  6e-04  2e-10  6e-15
10: -3.9308e+02 -3.9308e+02  2e-05  4e-12  6e-15
Optimal solution found.
140.23808908462524


In [ ]:
# find train and test accuracies
predicted_train=svm.predict(x_train)
predicted = svm.predict(x_test)
per1 = calc_percentage(predicted_train, y_train)
per2 = calc_percentage(predicted, y_test)
print("Train accuracy: ", per1)
print("Test accuracy: ", per2)

Train accuracy:  0.9946666666666667
Test accuracy:  0.976


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

start = time.time()
accuracies = []

# loop over various values of `k` for the k-Nearest Neighbor classifier
for k in range(1,4,2):
	# train the k-Nearest Neighbor classifier with the current value of `k`
	model = KNeighborsClassifier(n_neighbors=k)
	model.fit(x_train, y_train)

	# evaluate the model and update the accuracies list
	score = model.score(x_test, y_test)
	print("accuracy= ",  score)
	accuracies.append(score)
end = time.time()
print(end - start)

accuracy=  0.944
accuracy=  0.942
5.313709259033203


In [ ]:
from sklearn.neighbors import NearestCentroid

start = time.time()
model2 = NearestCentroid()
model2.fit(x_train, y_train)

score = model2.score(x_test, y_test)
print("Nearest Centroid, accuracy= ", score)
end = time.time()
print(end - start)

Nearest Centroid, accuracy=  0.802
0.01582956314086914
